Firsts, I import the libraries needed, and I read the file from the Wikipedia with pandas

In [16]:
import pandas as pd
import numpy as np
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
df = df[0]

Cleaning of the rows with no borough assigned.

In [17]:
df = df[df.Borough!='Not assigned']

In order to clean the repeated neighborhoods, I create a dictionary with each postal code as a key and its neighborhoods as its values.

In [18]:
codelists = list(df['Postal Code'].values)
codedict =  {}
for i, code in enumerate(codelists):
    if code in codedict.keys():
        codedict[code].append(df['Neighborhood'].values[i])
    else:
        codedict[code] = df['Neighborhood'].values[i]

Then, I create a dataframe with those two columns

In [19]:
df2 = pd.DataFrame()
df2['Postal Code'] = codedict.keys()
df2['Neighborhood'] = codedict.values()

I clean the original dataframe in order to do a left merge of both. Now I have the clean dataframe in df2.

In [20]:
df.drop('Neighborhood', axis=1,inplace=True)
df2 = df2.merge(df, on='Postal Code', how='left')
df2.head()

,Postal Code,Neighborhood,Borough
0,M3A,Parkwoods,North York
1,M4A,Victoria Village,North York
2,M5A,"Regent Park, Harbourfront",Downtown Toronto
3,M6A,"Lawrence Manor, Lawrence Heights",North York
4,M7A,"Queen's Park, Ontario Provincial Government",Downtown Toronto


It has 103 rows and 3 columns

In [21]:
df2.shape

(103, 3)

Merge the dataframe with the csv with coordinates provided

In [22]:
coords = pd.read_csv('https://cocl.us/Geospatial_data')
df2 = df2.merge(coords, on='Postal Code', how='left')
df2.head()

,Postal Code,Neighborhood,Borough,Latitude,Longitude
0,M3A,Parkwoods,North York,43.753259,-79.329656
1,M4A,Victoria Village,North York,43.725882,-79.315572
2,M5A,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636
3,M6A,"Lawrence Manor, Lawrence Heights",North York,43.718518,-79.464763
4,M7A,"Queen's Park, Ontario Provincial Government",Downtown Toronto,43.662301,-79.389494


In [82]:
df2.head()

,Postal Code,Neighborhood,Borough,Latitude,Longitude
0,M3A,Parkwoods,North York,43.753259,-79.329656
1,M4A,Victoria Village,North York,43.725882,-79.315572
2,M5A,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636
3,M6A,"Lawrence Manor, Lawrence Heights",North York,43.718518,-79.464763
4,M7A,"Queen's Park, Ontario Provincial Government",Downtown Toronto,43.662301,-79.389494


In [23]:
import folium
map_toronto = folium.Map(location=[43.7, -79.3], zoom_start=11)
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

In [25]:
map_toronto